### Demonstration Overview: Constructing the `Allele` Class

This notebook demonstrates the construction of the `Allele` class. The `Allele` class, built with Pydantic, extends the `MolecularDefinition` class to meet the specific requirements of the HL7 FHIR MolecularDefinition Allele. It incorporates tailored attributes and behaviors to support this use case. This notebook demonstrates how to use this class effectively, and shows how to build an Allele example step by step.


### Step 1: Prerequisites and Setup

To construct the `Allele` class in this demonstration, we set up the environment by importing the specific libraries and modules required for the examples in this notebook. These include:

1. **External Package**:
   - `Coding`, `CodeableConcept`, `Quantity`, `Reference`, and `Meta` from the `fhir.resources` library. These classes provide the framework for working with structured healthcare data in FHIR format.

2. **Custom Project Modules**:
   - `Allele` from `profiles.allele`: The main class we will build and demonstrate.
   - Components from `resources.moleculardefinition`:
     - `MolecularDefinitionLocation` and related classes for specifying molecular location.
     - `MolecularDefinitionRepresentation` and related classes for specifying  molecular representations.

By importing these libraries, we ensure access to all the tools needed for this demonstration.


In [6]:
# Import required libraries
from fhir.resources.coding import Coding
from fhir.resources.codeableconcept import CodeableConcept
from fhir.resources.quantity import Quantity
from fhir.resources.reference import Reference
from fhir.resources.meta import Meta
from profiles.allele import Allele as FhirAllele 
from resources.moleculardefinition import (
    MolecularDefinitionLocation,
    MolecularDefinitionLocationSequenceLocation,
    MolecularDefinitionLocationSequenceLocationCoordinateInterval,
    MolecularDefinitionLocationSequenceLocationCoordinateIntervalCoordinateSystem,
    MolecularDefinitionRepresentation,
    MolecularDefinitionRepresentationLiteral
)


### Step 2: Defining the Allele Location Component

In this step, we define the location component for the `Allele`, which is a subclass of the `MolecularDefinition`. 

- One necessary modification was adjusting the cardinality of the `location` attribute from `0..*` to `1..1` to ensure a mandatory and singular location specification.

#### Example Workflow:
The JSON location is implemented in Python as follows in next cell:
```json
{
  "location": [
    {
      "sequenceLocation": {
        "sequenceContext": {
          "reference": "MolecularDefinition/example-sequence-nc000002-url",
          "type": "MolecularDefinition",
          "display": "NC_000002.12"
        },
        "coordinateInterval": {
          "coordinateSystem": {
            "system": {
              "coding": [
                {
                  "system": "http://loinc.org",
                  "code": "LA30100-4",
                  "display": "0-based interval counting"
                }
              ],
              "text": "0-based interval counting"
            }
          },
          "startQuantity": {
            "value": 27453448
          },
          "endQuantity": {
            "value": 27453449
          }
        }
      }
    }
  ]
}

In [7]:
# Define the coding for the coordinateSystem
coding_val = Coding(
    system="http://loinc.org",
    code="LA30100-4",
    display="0-based interval counting",
)

# Wrap coding into the  CodeableConcept object
codeconcept_val = CodeableConcept(
    coding=[coding_val],
    text="0-based interval counting",
)

# Wrap the CodeableConcept into coordinateSystem for location
MolDefLocSeqLocCoordIntCoord = MolecularDefinitionLocationSequenceLocationCoordinateIntervalCoordinateSystem(
    system=codeconcept_val
)
# Create the coordinateInterval and build the interval with start and end quantities.
MolDefLocSeqLocCoordInt = MolecularDefinitionLocationSequenceLocationCoordinateInterval(
    coordinateSystem=MolDefLocSeqLocCoordIntCoord,
    startQuantity=Quantity(value=27453448),
    endQuantity=Quantity(value=27453449),
)
# Wrap the coordianteInterval into the sequenceLocation and add the sequenceContext.
MolDefLocSeqLoc = MolecularDefinitionLocationSequenceLocation(
    sequenceContext=Reference(
        reference="MolecularDefinition/example-sequence-nc000002-url",
        type="MolecularDefinition",
        display="NC_000002.12",
    ),
    coordinateInterval=MolDefLocSeqLocCoordInt,
)
# Create the MolecularDefinitionLocation object, including the sequenceLocation
MolDefLoc = MolecularDefinitionLocation(
    sequenceLocation=MolDefLocSeqLoc
)

# Inspect serialized output
MolDefLoc.model_dump()


{'sequenceLocation': {'sequenceContext': {'reference': 'MolecularDefinition/example-sequence-nc000002-url',
   'type': 'MolecularDefinition',
   'display': 'NC_000002.12'},
  'coordinateInterval': {'coordinateSystem': {'system': {'coding': [{'system': 'http://loinc.org',
       'code': 'LA30100-4',
       'display': '0-based interval counting'}],
     'text': '0-based interval counting'}},
   'startQuantity': {'value': 27453448.0},
   'endQuantity': {'value': 27453449.0}}}}

### Step 3: Defining the Allele Representation Component

In this step, we define the representation component for the `Allele`, which is a subclass of the `MolecularDefinition`. 

- One necessary modification was adjusting the cardinality of the `representation` attribute from `0..*` to `1..1` to ensure a mandatory and singular representation specification.  
- Additionally, the cardinality of `representation.focus.coding.code` was updated from `0..1` to `1..1`, with a default value of `allele-state`.  
- Similarly, the cardinality of `representation.focus.coding.display` was changed from `0..1` to `1..1`.

#### JSON Representation:
```json
{
  "representation": [
    {
      "focus": {
        "coding": [
          {
            "system": "http://hl7.org/fhir/moleculardefinition-focus",
            "code": "allele-state",
            "display": "Allele State"
          }
        ]
      },
      "literal": {
        "value": "T"
      }
    }
  ]
}


In [8]:
# Create a Coding instance to specify the focus.  
focus_coding_val = Coding(
        system="http://hl7.org/fhir/moleculardefinition-focus",
        code= "allele-state",
        display= "Allele State"
    )
# Wrapt the coding value into the CodeableConcept.
focus_codeconcept_val = CodeableConcept(
    coding=[focus_coding_val],
)

#Define the literal value using MolecularDefinitionRepresentationLiteral.
MolDefRepLit = MolecularDefinitionRepresentationLiteral(value="T")

# Integrate these into the MolecularDefinitionRepresentation. 
MolDefRep = MolecularDefinitionRepresentation(
    literal=MolDefRepLit,
    focus=focus_codeconcept_val
    )
# Inspect serialized output
MolDefRep.model_dump()


{'focus': {'coding': [{'system': 'http://hl7.org/fhir/moleculardefinition-focus',
    'code': 'allele-state',
    'display': 'Allele State'}]},
 'literal': {'value': 'T'}}

### Step 4: Integrating Components into the Complete `Allele`

In this step, we combine all the previously defined components to create a complete instance of the `Allele` class. This integration includes meta data including (resourceType, id, meta, moleculeType), location and representation components.

- One necessary modification was adjusting the cardinality of the `moleculeType` attribute from `0..1` to `1..1` to ensure a mandatory and singular specification.  
- Additionally, the `memberState` attribute has been removed as it is not required in the `Allele`.  
 
#### JSON Representation:
```json
{
  "resourceType": "MolecularDefinition",
  "id": "demo-example-hgvs-substitution",
  "meta": {
    "profile": ["http://hl7.org/fhir/StructureDefinition/allelesliced"]
  },
  "moleculeType": {
    "coding": [
      {
        "system": "http://hl7.org/fhir/sequence-type",
        "code": "dna",
        "display": "DNA Sequence"
      }
    ]
  },
  "location": [
    {...}  // Details from MolecularDefinitionLocation
  ],
  "representation": [
    {...}  // Details from MolecularDefinitionRepresentation
  ]
}


In [9]:
# Spcifies the profile URL in the Meta data type
meta_val = Meta(
  profile= ["http://hl7.org/fhir/StructureDefinition/allelesliced"]
)

# Defines the type of sequence, such as dna, rna, or aa.
moltype_coding_val = Coding(
        system="http://hl7.org/fhir/sequence-type",
        code= "dna",
        display= "DNA Sequence"
    )

# Wrap the moleculeType in the CodeableConcept data type.
moltype_codeconcept_val = CodeableConcept(
    coding=[moltype_coding_val],
)
#Incorporates the Location and Representation components created earlier.
Allele_Example = FhirAllele(
  id = 'demo-example-hgvs-substitution',
  meta = meta_val,
  moleculeType=moltype_codeconcept_val,
  location=[MolDefLoc],
  representation=[MolDefRep]
)

# Inspect serialized output
Allele_Example.model_dump()


{'resourceType': 'MolecularDefinition',
 'id': 'demo-example-hgvs-substitution',
 'meta': {'profile': ['http://hl7.org/fhir/StructureDefinition/allelesliced']},
 'moleculeType': {'coding': [{'system': 'http://hl7.org/fhir/sequence-type',
    'code': 'dna',
    'display': 'DNA Sequence'}]},
 'location': [{'sequenceLocation': {'sequenceContext': {'reference': 'MolecularDefinition/example-sequence-nc000002-url',
     'type': 'MolecularDefinition',
     'display': 'NC_000002.12'},
    'coordinateInterval': {'coordinateSystem': {'system': {'coding': [{'system': 'http://loinc.org',
         'code': 'LA30100-4',
         'display': '0-based interval counting'}],
       'text': '0-based interval counting'}},
     'startQuantity': {'value': 27453448.0},
     'endQuantity': {'value': 27453449.0}}}}],
 'representation': [{'focus': {'coding': [{'system': 'http://hl7.org/fhir/moleculardefinition-focus',
      'code': 'allele-state',
      'display': 'Allele State'}]},
   'literal': {'value': 'T'}}]

### Final Example: Using JSON to Generate the Class

Constructing the `Allele` step by step can be a bit challenging, especially if you are new to the `MolecularDefinition` schema. To ensure accuracy while constructing the `Allele`, we recommend having the schema available as a reference for guidance. 

Alternatively, instead of constructing the `Allele` step by step, you can use Python's `**` unpacking syntax to directly generate the class instance from the complete JSON structure. This method simplifies the process and ensures the class instance accurately reflects the provided JSON data.


In [10]:
# Example 1 - substitution origin: "NC_000002.12:g.27453449C>T"
example_substitution ={
    "resourceType" : "MolecularDefinition",
    "id" : "demo-example-hgvs-substitution",
    "meta" : {
      "profile" : ["http://hl7.org/fhir/StructureDefinition/allelesliced"]
    },
    "moleculeType" : {
      "coding" : [{
        "system" : "http://hl7.org/fhir/sequence-type",
        "code" : "dna",
        "display" : "DNA Sequence"
      }]
    },
    "location" : [
        {
      "sequenceLocation" : {
        "sequenceContext" : {
          "reference" : "MolecularDefinition/example-sequence-nc000002-url",
          "type" : "MolecularDefinition",
          "display" : "NC_000002.12"
        },
        "coordinateInterval" : {
          "coordinateSystem" : {
            "system" : {
              "coding" : [{
                "system" : "http://loinc.org",
                "code" : "LA30100-4",
                "display" : "0-based interval counting" 
              }],
              "text" : "0-based interval counting"
            }
          },
          "startQuantity" : {
            "value" : 27453448
          },
          "endQuantity" : {
            "value" : 27453449
          }
        }
      }
    }
    ],
    "representation" : [{
      "focus" : {
        "coding" : [{
          "system" : "http://hl7.org/fhir/moleculardefinition-focus",
          "code" : "allele-state",
          "display" : "Allele State"
        }]
      },
      "literal" : {
        "value" : "T"
      }
    }]
  }

FhirAllele(**example_substitution).model_dump()

{'resourceType': 'MolecularDefinition',
 'id': 'demo-example-hgvs-substitution',
 'meta': {'profile': ['http://hl7.org/fhir/StructureDefinition/allelesliced']},
 'moleculeType': {'coding': [{'system': 'http://hl7.org/fhir/sequence-type',
    'code': 'dna',
    'display': 'DNA Sequence'}]},
 'location': [{'sequenceLocation': {'sequenceContext': {'reference': 'MolecularDefinition/example-sequence-nc000002-url',
     'type': 'MolecularDefinition',
     'display': 'NC_000002.12'},
    'coordinateInterval': {'coordinateSystem': {'system': {'coding': [{'system': 'http://loinc.org',
         'code': 'LA30100-4',
         'display': '0-based interval counting'}],
       'text': '0-based interval counting'}},
     'startQuantity': {'value': 27453448.0},
     'endQuantity': {'value': 27453449.0}}}}],
 'representation': [{'focus': {'coding': [{'system': 'http://hl7.org/fhir/moleculardefinition-focus',
      'code': 'allele-state',
      'display': 'Allele State'}]},
   'literal': {'value': 'T'}}]

### Conclusion

The `Allele` resource is currently under development. For more details and the latest updates, please refer to the [HL7 FHIR Allele Documentation](https://build.fhir.org/branches/cg-im-moldef_work_in_progress_2/allelesliced.html).

**Note:** The documentation may occasionally experience downtime, which is beyond the control of our lab group. We appreciate your patience and apologize for any inconvenience this may cause.
